In [1]:
import gdrive
#install googledrivedownloader if gdrive error
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
shapely.speedups.enable()
import geopandas
from geopandas import GeoDataFrame

In [2]:
#download cluster file from gdrve https://drive.google.com/file/d/1KT0eZvx3ila1_H0W5rzEocY7UJJPwsIE/view?usp=sharing
gdrive.download_drive(file_id='1TBqz1x1qQEZGazwOPu0BnLgoo_G4j736',
                   dest_path = 'data/clusters_priority.xlsx')

In [3]:
#download LGA shape file from https://drive.google.com/file/d/1vq22_NO2yXB57MB4hUYcN4wXqFiygvqw/view?usp=sharing
gdrive.download_drive(file_id = '1vq22_NO2yXB57MB4hUYcN4wXqFiygvqw',
                     dest_path = 'data/nga_adm_osgof_20190417_shp.zip',
                     unzip=True)

In [4]:
clusters = pd.read_excel("data/clusters_priority.xlsx")
clusters.tail()

,label,population,lat,lon,area,density,address0,geometry,ongrid,score
1979,877,220979,11.707868,8.865077,694,318,Kafin Malamai,POINT (8.865076786724444 11.7078679976377),False,318
1980,1901,4948,9.024628,9.485341,15,318,Pyapun,POINT (9.485341313668878 9.024628225491194),False,318
1981,453,1914,12.472429,8.760248,6,315,Kaffi,POINT (8.760247598534551 12.47242872280043),False,315
1982,1258,6005,11.232167,9.195339,19,309,Kafin Lemo,POINT (9.195338519402725 11.23216682302277),False,309
1983,603,668,12.264122,7.931464,2,306,Yanganau,POINT (7.931463946786041 12.26412177383059),False,306


In [5]:
points = []
for lat, lon in zip(clusters['lat'],clusters['lon']):
    p = Point(lon,lat)
    points.append(p)

In [6]:
poly = geopandas.GeoDataFrame.from_file('data/nga_adm_osgof_20190417_shp/nga_admbnda_adm2_osgof_20190417.shp')
poly = poly[['ADM1_EN','ADM2_EN','geometry']]
poly

,ADM1_EN,ADM2_EN,geometry
0,Abia,Aba North,"POLYGON ((7.40111 5.08195, 7.40013 5.08237, 7...."
1,Abia,Aba South,"POLYGON ((7.38749 5.08275, 7.38632 5.08236, 7...."
2,Borno,Abadam,"POLYGON ((13.83477 13.42336, 13.81909 13.42175..."
3,Federal Capital Territory,Abaji,"POLYGON ((7.04587 9.23050, 7.02654 9.22216, 7...."
4,Akwa Ibom,Abak,"POLYGON ((7.81124 5.09453, 7.81233 5.09367, 7...."
...,...,...,...
769,Kaduna,Zango-Kataf,"POLYGON ((8.51617 9.73717, 8.51729 9.73228, 8...."
770,Kaduna,Zaria,"POLYGON ((7.77154 11.12362, 7.77156 11.11693, ..."
771,Taraba,Zing,"POLYGON ((11.78946 8.64357, 11.76143 8.66650, ..."
772,Zamfara,Zurmi,"POLYGON ((7.04768 12.99911, 7.04995 12.97205, ..."


In [7]:
location = []
for point in points:
    for adm1,adm2,po in zip(poly['ADM1_EN'],poly['ADM2_EN'],poly['geometry']):
        if po.contains(point)==True:
            adm = adm2+','+adm1
            location.append(adm)

In [8]:
location = pd.DataFrame(location)
location.columns = ['adm']

In [9]:
clusters = clusters.merge(location, left_index=True, right_index=True, how='inner')

In [10]:
clusters.head()

,label,population,lat,lon,area,density,address0,geometry,ongrid,score,adm
0,353,4222,12.673658,10.412106,0,17878,Gogiya,POINT (10.41210645432109 12.67365755797058),False,17878,"Guri,Jigawa"
1,359,9434,12.656897,10.342335,0,16520,Marma,POINT (10.34233549943618 12.65689708045506),False,16520,"Kiri Kasamma,Jigawa"
2,455,4308,12.472996,10.442321,0,15202,Musari,POINT (10.44232140128945 12.47299618311559),False,15202,"Guri,Jigawa"
3,1454,8710,10.731385,8.586353,0,14594,Doguwa,POINT (8.58635279403952 10.73138543975388),False,14594,"Doguwa,Kano"
4,241,5098,12.840322,10.908617,0,13806,Dawayo,POINT (10.908617395084 12.84032211959036),False,13806,"Bade,Yobe"


In [11]:
clusters.to_excel("data/clusters_priority_adm2.xlsx",index=False)

In [12]:
clusterbyadm = clusters.groupby(['adm'])[['population']].sum().reset_index()
clusterbyadm.sort_values(by='population',ascending=False)

,adm,population
55,"Bichi,Kano",339729
59,"Birni Kudu,Jigawa",304213
169,"Ikwo,Ebonyi",267677
276,"Minjibir,Kano",255115
388,"Wudil,Kano",249371
...,...,...
30,"Asa,Kwara",4043
103,"Esan North East,Edo",4014
308,"Okigwe,Imo",3712
266,"Markafi,Kaduna",2989


In [16]:
clusters[['population','area','density']].describe()

,population,area,density
count,1984.000000,1984.000000,1984.000000
mean,9998.417339,11.369960,1594.900706
std,16394.107489,31.728602,1782.875580
min,668.000000,0.000000,306.000000
25%,4988.000000,3.000000,693.000000
50%,6558.000000,6.000000,1011.500000
75%,9845.500000,12.000000,1697.000000
max,339729.000000,706.000000,17878.000000
